In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [13]:
train_data = pd.read_csv('train.csv')
train_data = train_data[['text','target']]
test_data = pd.read_csv('test.csv')
test_data = test_data[['text']]

In [10]:
# Not a disaster
train_data[train_data["target"] == 0]["text"].values[1]

'I love fruits'

In [11]:
# Is a disaster
train_data[train_data["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [17]:
test_data.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [16]:
train_data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


Filtering the tweets so only valid texts and words remain. Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [18]:
train_data['text'] = train_data['text'].apply(lambda x: x.lower())
train_data['text'] = train_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [19]:
for idx,row in train_data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [22]:
print(train_data[train_data['target'] == 0].size)
print(train_data[train_data['target'] == 1].size)

8684
6542


In [21]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_data['text'].values)
X = tokenizer.texts_to_sequences(train_data['text'].values)
X = pad_sequences(X)

Next, I compose the LSTM Network. Note that embed_dim, lstm_out, batch_size, dropout_x variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [33]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 128)           256000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 30, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
Y = pd.get_dummies(train_data['target']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5709, 30) (5709, 2)
(1904, 30) (1904, 2)


In [35]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

/Users/paritoshkalla/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
 - 5s - loss: 0.5636 - accuracy: 0.7077
Epoch 2/7
 - 4s - loss: 0.4212 - accuracy: 0.8112
Epoch 3/7
 - 5s - loss: 0.3762 - accuracy: 0.8352
Epoch 4/7
 - 6s - loss: 0.3400 - accuracy: 0.8551
Epoch 5/7
 - 5s - loss: 0.3108 - accuracy: 0.8658
Epoch 6/7
 - 5s - loss: 0.2826 - accuracy: 0.8763
Epoch 7/7
 - 5s - loss: 0.2574 - accuracy: 0.8928


Extracting a validation set, and measuring score and accuracy.

In [36]:
validation_size = 750

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.60
acc: 0.76


In [37]:
dis_cnt, ndis_cnt, dis_correct, ndis_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            ndis_correct += 1
        else:
            dis_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        ndis_cnt += 1
    else:
        dis_cnt += 1
print("dis_acc", dis_correct/dis_cnt*100, "%")
print("ndis_acc", ndis_correct/ndis_cnt*100, "%")

dis_acc 66.45569620253164 %
ndis_acc 90.32258064516128 %


In [50]:
tokenizer.fit_on_texts(test_data['text'].values)

In [51]:
test_vectors = tokenizer.texts_to_sequences(test_data['text'].values)

In [74]:
test_vectors = pad_sequences(test_vectors, maxlen=30, padding='post', truncating='post')

In [105]:
sample_submission = pd.read_csv('sample_submission.csv')

In [106]:
sample_submission.shape

(3263, 2)

In [92]:
probabilities = model.predict(test_vectors)

In [108]:
probabilities

array([[0.00686583, 0.99313414],
       [0.85874164, 0.14125837],
       [0.7117328 , 0.28826723],
       ...,
       [0.47601444, 0.5239855 ],
       [0.6506684 , 0.3493316 ],
       [0.69057506, 0.3094249 ]], dtype=float32)

In [109]:
sample_submission['target'] = probabilities

In [110]:
sample_submission.head()

,id,target
0,0,0.006866
1,2,0.858742
2,3,0.711733
3,9,0.734360
4,11,0.307572


In [111]:
sample_submission['target'] = np.where(sample_submission['target']>0.5, 1, 0)

In [112]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,0


In [113]:
pd.value_counts(sample_submission['target'])

1    2441
0     822
Name: target, dtype: int64

In [114]:
sample_submission.to_csv("submission.csv", index=False)